# リポジトリクローン(Clone the repository)

In [ ]:
!git clone https://github.com/Kazuhito00/Multimodal-Node-Editor
%cd Multimodal-Node-Editor

# Pythonパッケージインストール(Install Python packages)

In [ ]:
!pip install -r requirements-gpu.txt

# モデルウェイトダウンロード(Download model weights)
このノートブックではモデル単体の最大ダウンロードサイズを150MBに制限しています(This notebook limits the maximum download size of a single model to 150 MB. Please adjust the option as needed.)

In [ ]:
!python download_weights.py --max-size 150

# npmパッケージインストール & ビルド<br>(Install and build npm packages)

In [ ]:
!npm --version

In [ ]:
%cd src/gui/reactflow/frontend
!npm ci
!npm run build
%cd ../../../../

# コンフィグ設定(Config settings)

In [6]:
!cp config.example.json config.json

In [7]:
import json
from pathlib import Path
try:
    from google.colab import userdata
except ImportError:
    userdata = None

# 読み込み
config_path = Path("config.json")
with config_path.open("r", encoding="utf-8") as f:
    config = json.load(f)

# オートダウンロード 設定
auto_download = config.setdefault("auto_download", {})
auto_download["video"] = True
auto_download["wav"] = True
auto_download["capture"] = True
auto_download["text"] = True

# OPENAI_API_KEY を Colab シークレット から設定
openai_api_key = None
if userdata is not None:
    try:
        openai_api_key = userdata.get("OPENAI_API_KEY")
    except KeyError:
        pass
if openai_api_key:
    api_keys = config.setdefault("api_keys", {})
    api_keys["openai"] = openai_api_key

# 保存
with config_path.open("w", encoding="utf-8") as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

# 起動(Launch the application)

In [8]:
# バックエンドをバックグラウンド起動
!uvicorn src.gui.reactflow.backend.main:app --host 0.0.0.0 --port 8000 > uvicorn.log 2>&1 &
!sleep 10 &

In [ ]:
from google.colab import output

output.serve_kernel_port_as_window(8000)  # 別ウィンドウ表示 ※動作しないときはserve_kernel_port_as_iframe()を試す
# output.serve_kernel_port_as_iframe(8000, height=1080)  # セル出力に表示(コードセルの出力アクション(…)から「出力を全画面表示」を選択)